# Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import panel as pn
import panel.widgets as pnw

pn.extension("plotly", "tabulator", theme='dark')
pn.extension()

# Data Loading


In [ ]:
# transactions data
transactions = pd.read_csv('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Data/data/transactions_cleaned.csv')
transactions['Transaction Date'] = pd.to_datetime(transactions['Transaction Date'])
transactions['Publication Date'] = pd.to_datetime(transactions['Publication Date'])

In [ ]:
# trades data
trades = pd.read_csv('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Data/data/trades.csv')

In [ ]:
# more prices data
# future and past prices fromt the dates of transactions
more_prices = pd.read_csv('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Data/data/more_prices_processed.csv')

In [ ]:
# global vars
party_colors = {
        'Democrat': '#0015BC',
        'Republican': '#DE0100',
        'Other': '#777777',
    }

info_by_name = transactions[['Politician Name', 'Party', 'Chamber', 'State']].drop_duplicates()
info_by_name = info_by_name.merge(transactions.groupby('Politician Name').size().reset_index(name='Total Transactions'), on="Politician Name")
info_by_name = info_by_name.merge(trades.groupby('Politician Name').size().reset_index(name='Total Trade Cycles'), on="Politician Name")

# Transaction Count Analysis Tab

In [ ]:
# transactions tab
group_by_selector = pnw.Select(options=['Politician Name', 'Party', 'Chamber', 'State', 'Issuer Name'])

plot_type_selector = pn.widgets.RadioButtonGroup(options=["Bar Plot", "Pie Chart"])
plot_functions = {"Bar Plot": px.bar, "Pie Chart": px.pie}

show_entries_count = pnw.EditableIntSlider(start=10, end=169, step=1, value=10)


def update_transactions_plot(event):
    col = group_by_selector.value
    plot_func = plot_functions[plot_type_selector.value]
    # data = info_by_name.sort_values('Total Transactions').tail(show_entries_count.value)
    data = transactions.groupby(col).size().reset_index(name='Total Transactions').sort_values('Total Transactions').tail(show_entries_count.value)

    if plot_type_selector.value == 'Pie Chart':
        if col == 'Politician Name':
            data = data.merge(info_by_name.drop(['Total Transactions'], axis=1), on='Politician Name')

            fig = plot_func(data, names='Politician Name', values='Total Transactions', color='Party', color_discrete_map=party_colors)

        elif col == 'Party':
            fig = plot_func(data, names='Party', values='Total Transactions', color='Party', color_discrete_map=party_colors)
        else:
            fig = plot_func(data, names=col, values='Total Transactions')

    else:
        if col == 'Politician Name':
            data = data.merge(info_by_name.drop(['Total Transactions'], axis=1), on='Politician Name')

            fig = plot_func(data, x='Total Transactions', y='Politician Name', hover_data=data.columns, color='Party', color_discrete_map=party_colors)
        elif col == 'Party':
            fig = plot_func(data, x='Total Transactions', y='Party', hover_data=data.columns, color='Party', color_discrete_map=party_colors)
        else:
            fig = plot_func(data, x=data['Total Transactions'], y=data.columns[0])

        fig.update_yaxes(categoryorder='total ascending')

    fig.update_layout(
        title={
            'text': "Distribution of Transaction Count by " + col,
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )

    transactions_plot_pane.object = fig


group_by_selector.param.watch(update_transactions_plot, 'value')
plot_type_selector.param.watch(update_transactions_plot, 'value')
show_entries_count.param.watch(update_transactions_plot, 'value')

transactions_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width', height=600)

update_transactions_plot(None)

transaction_count_analysis_tab = pn.Column(pn.Row("# Transaction Count Analysis"),
                                           pn.Row("Group by", group_by_selector),
                                           pn.Row("Show Top", show_entries_count),
                                           pn.Row("Plot Type", plot_type_selector),
                                           transactions_plot_pane,
                                           name= 'Transaction Count Analysis'
                                          )
# TODO: Fix Pie Chart Hover Data
# Add list of politicians to include
# Add reverse order button
# add checkboxes for parameters (name, party, state, chamber)

# Trade Cycle Analysis Tab

In [ ]:
# trades tab
trades_tab_selector = pn.widgets.RadioButtonGroup(options=["Trades Profitability Distribution", "Average Profit by Politician", "Profitability Scatterplot"])


# axis scale slider
range_slider = pnw.RangeSlider()
min_value_input = pnw.IntInput()
max_value_input = pnw.IntInput()
update_range_button = pnw.Button()

def update_slider_from_inputs(event):
    range_slider.value = (int(min_value_input.value), int(max_value_input.value))

def update_inputs_from_slider(event):
    min_value_input.value, max_value_input.value = int(range_slider.value[0]), int(range_slider.value[1])

def update_slider_range(min_val, max_val, step):

    range_slider.start = min_val
    range_slider.end = max_val
    range_slider.step = step
    range_slider.value = (min_val, max_val)

    min_value_input.start = min_val
    min_value_input.end = max_val
    min_value_input.step = step
    min_value_input.value = min_val

    max_value_input.start = min_val
    max_value_input.end = max_val
    max_value_input.step = step
    max_value_input.value = max_val

def update_slider_name(min_name, max_name, but_name):
    min_value_input.name = min_name
    max_value_input.name = max_name
    update_range_button.name = but_name


# highlight_sus init
highlight_sus_trades_button = pnw.Button(name='Zoom In Suspicious Politicians')

highlight_sus_trades = False

def highlight_sus_trades_click(event):
    global highlight_sus_trades
    highlight_sus_trades = not highlight_sus_trades
    update_trades_plot(event)
    if highlight_sus_trades:
        highlight_sus_trades_button.name = 'Zoom Out Suspicious Politicians'
    else:
        highlight_sus_trades_button.name = 'Zoom In Suspicious Politicians'

# plot updates
def update_trades_plot(event):
    if trades_tab_selector.value == 'Trades Profitability Distribution':
        if event is not None and event.name != 'clicks': #update range button was pressed. do not reset the range
            update_slider_range(-2900, 3200, 1)
            update_slider_name('Min Value (x)', 'Max Value (x)', 'Update Range (x-axis)')

        # data = trades[(trades['Annualized Percentage Profit'] >= min_value_input.value) & (trades['Annualized Percentage Profit'] <= max_value_input.value)]
        data = pd.DataFrame()

        if highlight_sus_trades:
            data = trades[trades['Annualized Percentage Profit'] >= 20]
        else:
            data = trades[(trades['Annualized Percentage Profit'] >= min_value_input.value) & (trades['Annualized Percentage Profit'] <= max_value_input.value)]

        fig = px.histogram(data, x='Annualized Percentage Profit', color_discrete_sequence=['#1B4242'])

    elif trades_tab_selector.value == 'Average Profit by Politician':
        if event is not None and event.name != 'clicks': #update range button was pressed. do not reset the range
            update_slider_range(1, 78, 1)
            update_slider_name('Min Value (y)', 'Max Value (y)', 'Update Range (y-axis)')

        data = trades.groupby('Politician Name')['Annualized Percentage Profit'].mean().sort_values(ascending=False).reset_index(name='Average Annualized Percentage Profit')
        data = data.merge(transactions[['Politician Name', 'Party', 'Chamber', 'State']].drop_duplicates(), on='Politician Name')
        data = data.merge(trades.groupby('Politician Name').size().reset_index(name='Total Trade Cycles').drop_duplicates(), on='Politician Name')
        data = data.iloc[min_value_input.value - 1:max_value_input.value]


        if highlight_sus_trades:
            data = data[(data['Average Annualized Percentage Profit'] >= 20)]
        else:
            data = data.iloc[min_value_input.value - 1:max_value_input.value]

        fig = px.bar(data, x='Average Annualized Percentage Profit', y='Politician Name', hover_data=data.columns, color='Party', color_discrete_map=party_colors)
        fig.update_yaxes(categoryorder='total ascending')
    else:
        if event is not None and event.name != 'clicks': #update range button was pressed. do not reset the range
            update_slider_range(-260, 700, 1)
            update_slider_name('Min Value (x)', 'Max Value (x)', 'Update Range (x-axis)')

        data = trades.groupby('Politician Name')['Annualized Percentage Profit'].mean().sort_values(ascending=False).reset_index(name='Average Annualized Percentage Profit')
        data = data.merge(info_by_name, on='Politician Name')

        if highlight_sus_trades:
            data = data[(data['Average Annualized Percentage Profit'] >= 20)]
        else:
            data = data[(data['Average Annualized Percentage Profit'] >= min_value_input.value) & (data['Average Annualized Percentage Profit'] <= max_value_input.value)]

        fig = px.scatter(data, x='Average Annualized Percentage Profit', y='Total Trade Cycles', hover_data=data.columns, color='Party', color_discrete_map=party_colors)

    fig.update_layout(
        title={
            'text': trades_tab_selector.value,
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )
    trades_plot_pane.object = fig


# dynamic updates
trades_tab_selector.param.watch(update_trades_plot, 'value')

min_value_input.param.watch(update_slider_from_inputs, 'value')
max_value_input.param.watch(update_slider_from_inputs, 'value')
range_slider.param.watch(update_inputs_from_slider, 'value')
update_range_button.on_click(update_trades_plot)
highlight_sus_trades_button.on_click(highlight_sus_trades_click)

# init
trades_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width', height=600)
update_slider_range(-2900, 3200, 1)
update_slider_name('Min Value (x)', 'Max Value (x)', 'Update Range (x-axis)')
update_trades_plot(None)


# group
trade_cycle_analysis_tab = pn.Column(pn.Row("# Trade Cycle Analysis"),
                                     pn.Row(trades_tab_selector),
                                     pn.Row(min_value_input, range_slider, max_value_input, pn.Column(pn.Spacer(height=17), update_range_button), pn.Spacer(width=50), 'OR', pn.Spacer(width=50), pn.Column(pn.Spacer(height=17), highlight_sus_trades_button)),
                                     trades_plot_pane,
                                     name= 'Trade Cycle Analysis'
                                      )
# todo
# add checkbox for parameters (name, party, state, chamber)

# Decision Analysis Tab

In [ ]:
# decision tab
# data
politician_decisions = more_prices.groupby('Politician Name')['transaction_eval'].value_counts().reset_index(name='Decision Counts')
politician_decisions = politician_decisions.merge(info_by_name, on = 'Politician Name')
politician_decisions['Proportion of Decision'] = politician_decisions['Decision Counts'] / politician_decisions['Total Transactions'] * 100
politician_decisions['transaction_eval'] = politician_decisions['transaction_eval'].astype(str)


#
decision_type_selector = pn.widgets.CheckBoxGroup(name='Checklist',
                                                  options=['Very Good Decision', 'Good Decision', 'Bad Decision', 'Very Bad Decision'],
                                                  value=['Very Good Decision'],
                                                  inline=False)

# highlight_sus init
highlight_sus_button = pnw.Button(name='Zoom In Suspicious Politicians')

highlight_sus = False

def highlight_sus_click(event):
    global highlight_sus
    highlight_sus = not highlight_sus
    update_decision_plot(event)
    if highlight_sus:
        highlight_sus_button.name = 'Zoom Out Suspicious Politicians'
        decision_type_selector.value = ['Very Good Decision']
    else:
        highlight_sus_button.name = 'Zoom In Suspicious Politicians'


# plot updates
def update_decision_plot(event):
    data = politician_decisions[politician_decisions['transaction_eval'].isin(decision_type_selector.value)]
    if highlight_sus:
        data = politician_decisions[(politician_decisions['Proportion of Decision'] > 35) & (politician_decisions['transaction_eval'] == 'Very Good Decision')]

    fig = px.scatter(data, x='Proportion of Decision', y='Total Transactions', hover_data=data.columns, color='Party', color_discrete_map=party_colors, symbol='transaction_eval')

    fig.update_layout(
        title={
            'text': 'Proportion of' + str(decision_type_selector.value) + " VS Total Transactions",
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )
    decision_plot_pane.object = fig



# dynamic updates
highlight_sus_button.on_click(highlight_sus_click)
decision_type_selector.param.watch(update_decision_plot, 'value')

# init
decision_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width', height=600)
update_decision_plot(None)


# group
decision_analysis_tab = pn.Column(pn.Row("# Decision Analysis"),
                                  pn.Row(decision_type_selector, highlight_sus_button),
                                    decision_plot_pane,
                                    name= 'Decision Analysis'
                                 )

# Methodology Tab

In [ ]:
roadmap_pic = pn.pane.JPG('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Resources/Complete%20Roadmap.jpeg', sizing_mode='stretch_width', height=600)

cycle_analysis_pic = pn.pane.JPG('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Resources/cycle_analysis.png', sizing_mode='stretch_width', height=600)
cycle_definition_pic = pn.pane.JPG('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Resources/cycle_definition.png', sizing_mode='stretch_width', height=600)

transaction_evaluation_definition_pic = pn.pane.JPG('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Resources/transaction_evaluation_definition.png', sizing_mode='stretch_width', height=600)

transactions_info = """
Transactions were scraped from CapitolTrades.com. Subsequently, the stock prices
were updated using Yahoo Finance, as some entries contained incorrect prices.
Transactions reporting unknown tickers, cryptocurrencies, or hedge funds were
excluded.
"""
trades_info = """
Trades were formed using a cycle algorithm that grouped all transactions by
politician names and tickers. For each politician and ticker, the algorithm
generated trade cycles. A trade cycle starts with a series of stock purchases
and ends with a series of sales. The average purchase and sell prices are
calculated based on the mean of all buy and sell prices in the cycle.
The average days between transactions are computed as the difference between
the mean sell and purchase dates. From these two factors, the trade’s Annualized
Percentage Profit is calculated.
"""
decisions_info = """
Initially, for each trade, stock prices were accrued before and after
the transaction date using day offsets of
[-90, -60, -40, -20, -10, 0, 10, 20, 40, 60, 90] from the transaction date.
Based on this data, the average performance was
estimated before and after each purchase. These estimators allow for
categorizing all transactions into four figures:

1. Dip (down before the transaction occurred, up after)
2. Continuous Growth (up before and after the transaction occurred)
3. Continuous Decline (down before and after the transaction occurred)
4. Peak (up before the transaction occurred, down after)

After classifying each trade, an additional level of analysis was applied to
classify the success of the transactions:

- Continuous Growth BUY - Good Decision
- Continuous Growth SELL - Bad Decision
- Continuous Decline BUY - Bad Decision
- Continuous Decline SELL - Good Decision
- Peak BUY - Very Bad Decision
- Peak SELL - Very Good Decision
- Dip BUY - Very Good Decision
- Dip SELL - Very Bad Decision

Assumptions:

- Using a random trading strategy, the proportion of very bad, bad, good, and
  very good decisions will be equal.
- A professional trader does not achieve more than 35% of Very Good Decisions.
"""


methodology_tab = pn.Column(pn.Row("# Methodology"),
                            pn.layout.Divider(),
                            pn.Row(pn.Column("## Data Acquisition and Transformation Map", roadmap_pic)),
                            pn.layout.Divider(),
                            pn.Row(pn.Column("## About Transactions", transactions_info)),
                            pn.layout.Divider(),
                            pn.Row(pn.Column("## About Trades", trades_info), pn.Row(cycle_definition_pic, cycle_analysis_pic)),
                            pn.layout.Divider(),
                            pn.Row(pn.Column("## About Decisions", decisions_info), transaction_evaluation_definition_pic),
                            pn.layout.Divider(),
                            pn.Row(pn.layout.HSpacer(), "Made by Max Dokukin", pn.Spacer(width=100)),
                            name= 'Methodology'
                            )

# Dashboard Setup

In [ ]:
tabs = pn.Tabs(methodology_tab,
               transaction_count_analysis_tab,
               trade_cycle_analysis_tab,
               decision_analysis_tab
               )

link_html = """
<a href="https://github.com/maxdokukin/Politician-Trades/tree/main" target="_blank" style="text-decoration: none;">
    <button style='color: white; background-color: #1b73ba; border: none; padding: 8px 15px; text-align: center;
    display: inline-block; font-size: 16px; margin: 4px 2px; cursor: pointer; border-radius: 5px;'>
        GitHub
    </button>
</a>
"""

title_pane = pn.pane.Markdown("# Let's Investigate Politician Stock Trading Together!", width=700)
github_button = pn.pane.HTML(link_html, width=120, margin=(0,0,0,10))

full_dashboard = pn.Row(pn.Column(pn.Row(title_pane, pn.layout.HSpacer(), github_button),
                           "## Inquiry",
                           tabs
                        ))

# Dashboard Run

In [ ]:
full_dashboard.servable()
# Optimized for dark mode only

Row
    [0] Column
        [0] Row
            [0] Markdown(str, width=700)
            [1] HSpacer()
            [2] HTML(str, margin=(0, 0, 0, 10), width=120)
        [1] Markdown(str)
        [2] Tabs
            [0] Column(name='Methodology')
                [0] Row
                    [0] Markdown(str)
                [1] Divider()
                [2] Row
                    [0] Column
                        [0] Markdown(str)
                        [1] JPG(str, height=600, sizing_mode='stretch_width')
                [3] Divider()
                [4] Row
                    [0] Column
                        [0] Markdown(str)
                        [1] Markdown(str)
                [5] Divider()
                [6] Row
                    [0] Column
                        [0] Markdown(str)
                        [1] Markdown(str)
                    [1] Row
                        [0] JPG(str, height=600, sizing_mode='stretch_width')
                        [1] JPG(str, height=600, sizing_mode='stretch_width')
                [7] Divider()
                [8] Row
                    [0] Column
                        [0] Markdown(str)
                        [1] Markdown(str)
                    [1] JPG(str, height=600, sizing_mode='stretch_width')
                [9] Divider()
                [10] Row
                    [0] HSpacer()
                    [1] Markdown(str)
                    [2] Spacer(width=100)
            [1] Column(name='Transaction C...)
                [0] Row
                    [0] Markdown(str)
                [1] Row
                    [0] Markdown(str)
                    [1] Select(options=['Politician Name', ...], value='Politician Name')
                [2] Row
                    [0] Markdown(str)
                    [1] EditableIntSlider(end=169, start=10, value=10, width=300)
                [3] Row
                    [0] Markdown(str)
                    [1] RadioButtonGroup(options=['Bar Plot', 'Pie Chart'], value='Bar Plot')
                [4] Plotly(Figure, height=600, sizing_mode='stretch_width')
            [2] Column(name='Trade Cycle Analysis')
                [0] Row
                    [0] Markdown(str)
                [1] Row
                    [0] RadioButtonGroup(options=['Trades Profitability Dis...], value='Trades Profitability D...)
                [2] Row
                    [0] IntInput(end=3200, name='Min Value (x)', start=-2900, value=-2900)
                    [1] RangeSlider(end=3200, start=-2900, step=1, value=(-2900, 3200), value_end=3200, value_start=-2900)
                    [2] IntInput(end=3200, name='Max Value (x)', start=-2900, value=3200)
                    [3] Column
                        [0] Spacer(height=17)
                        [1] Button(name='Update Range (x-axis)')
                    [4] Spacer(width=50)
                    [5] Markdown(str)
                    [6] Spacer(width=50)
                    [7] Column
                        [0] Spacer(height=17)
                        [1] Button(name='Zoom In Suspicious P...)
                [3] Plotly(Figure, height=600, sizing_mode='stretch_width')
            [3] Column(name='Decision Analysis')
                [0] Row
                    [0] Markdown(str)
                [1] Row
                    [0] CheckBoxGroup(name='Checklist', options=['Very Good Decision', ...], value=['Very Good Decision'])
                    [1] Button(name='Zoom In Suspicious P...)
                [2] Plotly(Figure, height=600, sizing_mode='stretch_width')